In [28]:
#Importamos las librerias:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import recall_score



In [29]:
data = pd.read_csv('./hospitalizaciones_train.csv')

In [30]:
data

,Available Extra Rooms in Hospital,Department,Ward_Facility_Code,doctor_name,staff_available,patientid,Age,gender,Type of Admission,Severity of Illness,health_conditions,Visitors with Patient,Insurance,Admission_Deposit,Stay (in days)
0,4,gynecology,D,Dr Sophia,0,33070,41-50,Female,Trauma,Extreme,Diabetes,4,Yes,2966.408696,8
1,4,gynecology,B,Dr Sophia,2,34808,31-40,Female,Trauma,Minor,Heart disease,2,No,3554.835677,9
2,2,gynecology,B,Dr Sophia,8,44577,21-30,Female,Trauma,Extreme,Diabetes,2,Yes,5624.733654,7
3,4,gynecology,D,Dr Olivia,7,3695,31-40,Female,Urgent,Moderate,None,4,No,4814.149231,8
4,2,anesthesia,E,Dr Mark,10,108956,71-80,Male,Trauma,Moderate,Diabetes,2,No,5169.269637,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409995,1,gynecology,D,Dr Sarah,1,63105,31-40,Female,Trauma,Minor,Asthama,3,Yes,3902.291076,8
409996,5,gynecology,D,Dr Nathan,5,129292,21-30,Female,Trauma,Extreme,Other,4,No,4771.995223,9
409997,4,gynecology,D,Dr Sarah,9,11399,21-30,Female,Emergency,Minor,Asthama,4,Yes,3816.994210,9
409998,10,gynecology,D,Dr Olivia,10,75003,21-30,Female,Trauma,Moderate,Other,2,Yes,3841.577491,9


Paso 1.-De acuerdo a lo indicado, se crea una columna nueva llamada Prep

In [7]:
data['Prep']= data['Stay (in days)'].apply(lambda x:1 if x>8 else 0)
data= data.drop(['Stay (in days)'], axis=1)

Paso 2.- Correr metodos de seleccion de caracteristicas.-Importancia de la ingeniería de características
La ingeniería de características es un paso muy importante en el aprendizaje automático. 
Se correra correlacion de Pearson y CHI cuadrado, ya que tenemos muchas columnas categoricas en el modelo.
con el fin de escoger las columnas que hagan mas eficiente nuestro modelo, se diseña lo siguiente:
1.- Con las pruebas de CHI cuadrado y Pearson se diseñam los campos que tienen una mejor correlacion para nuestro modelo.
2.- Metodo de envoltura, se realiza los calculos con el metodo envoltura considerando todos los campos, y se revisa si los valores obtenidos son significativos con el modelo seleccionado en el paso 1.
Los modelos que se probaran son:
1.- Regresion Polinomial 
1.- Arboles de decision
2, K vecinos



In [8]:
from sklearn import preprocessing
escenario=data
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
escenario['Age_rango']= label_encoder.fit_transform(escenario['Age'])
escenario['health_conditions-valor']= label_encoder.fit_transform(escenario['health_conditions'])
escenario['Ward_Facility_Code-valor']= label_encoder.fit_transform(escenario['Ward_Facility_Code'])



In [9]:
escenario=pd.get_dummies(escenario)
escenario.columns


Index(['Available Extra Rooms in Hospital', 'staff_available', 'patientid',
       'Visitors with Patient', 'Admission_Deposit', 'Prep', 'Age_rango',
       'health_conditions-valor', 'Ward_Facility_Code-valor',
       'Department_TB & Chest disease', 'Department_anesthesia',
       'Department_gynecology', 'Department_radiotherapy',
       'Department_surgery', 'Ward_Facility_Code_A', 'Ward_Facility_Code_B',
       'Ward_Facility_Code_C', 'Ward_Facility_Code_D', 'Ward_Facility_Code_E',
       'Ward_Facility_Code_F', 'doctor_name_Dr Isaac', 'doctor_name_Dr John',
       'doctor_name_Dr Mark', 'doctor_name_Dr Nathan', 'doctor_name_Dr Olivia',
       'doctor_name_Dr Sam', 'doctor_name_Dr Sarah', 'doctor_name_Dr Simon',
       'doctor_name_Dr Sophia', 'Age_0-10', 'Age_11-20', 'Age_21-30',
       'Age_31-40', 'Age_41-50', 'Age_51-60', 'Age_61-70', 'Age_71-80',
       'Age_81-90', 'Age_91-100', 'gender_Female', 'gender_Male',
       'gender_Other', 'Type of Admission_Emergency',
       'Typ

In [10]:
escenario.corr()


,Available Extra Rooms in Hospital,staff_available,patientid,Visitors with Patient,Admission_Deposit,Prep,Age_rango,health_conditions-valor,Ward_Facility_Code-valor,Department_TB & Chest disease,...,Severity of Illness_Minor,Severity of Illness_Moderate,health_conditions_Asthama,health_conditions_Diabetes,health_conditions_Heart disease,health_conditions_High Blood Pressure,health_conditions_None,health_conditions_Other,Insurance_No,Insurance_Yes
Available Extra Rooms in Hospital,1.000000,-0.003064,-0.005426,0.071073,-0.050775,0.053894,-0.029921,0.001643,0.073566,-0.007705,...,0.023370,0.008172,-0.001906,-0.001202,0.000288,0.002685,-0.000194,0.000254,-0.002438,0.002438
staff_available,-0.003064,1.000000,-0.000350,0.001444,0.000816,0.002883,0.001818,-0.001500,-0.008233,-0.003403,...,-0.003376,0.003627,0.002259,-0.001979,0.001911,0.000112,0.000494,-0.001899,-0.001245,0.001245
patientid,-0.005426,-0.000350,1.000000,-0.006393,0.011602,-0.000532,0.009765,0.002306,-0.023729,0.002020,...,-0.001102,-0.008101,-0.000197,-0.001709,-0.002631,0.000012,0.001227,0.001686,-0.001827,0.001827
Visitors with Patient,0.071073,0.001444,-0.006393,1.000000,-0.069566,-0.009529,0.030136,-0.000117,0.075128,0.056122,...,-0.010011,0.027736,0.002645,-0.002017,-0.000939,-0.005999,0.007756,-0.003651,-0.025426,0.025426
Admission_Deposit,-0.050775,0.000816,0.011602,-0.069566,1.000000,-0.016650,0.016743,0.000412,-0.005523,0.015414,...,0.015841,-0.002684,-0.000334,0.000531,-0.000117,0.000424,-0.002374,0.002329,-0.001164,0.001164
Prep,0.053894,0.002883,-0.000532,-0.009529,-0.016650,1.000000,0.229657,-0.002568,-0.074337,0.154842,...,-0.037693,0.029866,0.005211,-0.003826,0.000423,0.006233,-0.007809,0.002432,0.001063,-0.001063
Age_rango,-0.029921,0.001818,0.009765,0.030136,0.016743,0.229657,1.000000,0.037140,-0.129855,0.192754,...,-0.064853,0.046796,-0.057827,0.019836,0.055281,-0.064272,0.030950,0.021685,0.002472,-0.002472
health_conditions-valor,0.001643,-0.001500,0.002306,-0.000117,0.000412,-0.002568,0.037140,1.000000,-0.000677,0.001077,...,0.019737,0.156516,-0.669964,-0.488253,-0.140340,-0.002467,0.417490,0.574981,-0.008702,0.008702
Ward_Facility_Code-valor,0.073566,-0.008233,-0.023729,0.075128,-0.005523,-0.074337,-0.129855,-0.000677,1.000000,0.015199,...,0.134295,0.030666,0.001975,-0.002186,0.002096,-0.000993,0.000601,-0.000689,-0.002697,0.002697
Department_TB & Chest disease,-0.007705,-0.003403,0.002020,0.056122,0.015414,0.154842,0.192754,0.001077,0.015199,1.000000,...,-0.010984,0.004699,-0.001433,0.001009,-0.000473,-0.000148,-0.000953,0.001846,-0.002240,0.002240


In [ ]:
#se elabora el pairplot
sns.pairplot(escenario, hue="Prep")
plt.show()

In [13]:
X=escenario.drop(['patientid','Prep'], axis=1)
y = escenario.Prep # Etiqueta a predecir

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=30)

# Instanciamos un objeto de la clase DecisionTreeClassifier

clf = DecisionTreeClassifier( max_depth =20,random_state = 30) 

In [15]:
# Entrenamos el modelo


clf.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=20, random_state=30)

In [16]:
# Exploramos algunas de las características del modelo entrenado. Para pensar, ¿qúe representa cada uno de estos valores?

print(clf.classes_)
print(clf.n_classes_)
print(clf.max_features_)
print(clf.feature_importances_)

[0 1]
2
54
[6.90713072e-02 2.75568344e-02 5.37306055e-02 2.20161106e-01
 3.33758549e-01 8.19154257e-03 9.05604267e-03 1.56673766e-04
 3.74123231e-04 2.95183580e-02 4.55326147e-02 0.00000000e+00
 1.27291271e-04 4.33153136e-03 3.02509834e-05 1.04023868e-02
 8.90206276e-05 9.73255350e-03 0.00000000e+00 0.00000000e+00
 6.61326412e-02 2.38310487e-03 1.99832546e-03 0.00000000e+00
 2.35144505e-03 2.23359367e-06 2.07018837e-03 0.00000000e+00
 2.50330025e-03 9.97025231e-03 9.70009656e-03 2.83601489e-03
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 5.04546982e-05 0.00000000e+00 6.24269994e-05
 9.35449802e-03 1.81868203e-02 6.51516349e-03 6.08816345e-03
 8.85697320e-03 1.08958728e-02 1.52858951e-03 1.88278921e-03
 1.94832443e-03 2.78921439e-03 3.40985350e-03 1.64284959e-03
 2.38892942e-03 2.63068269e-03]


In [17]:
# Predecimos

y_pred = clf.predict(X_test)

In [18]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test,y_pred))

0.7757235772357723


In [21]:

recall_score(y_test, y_pred)

0.8225530971720623

In [20]:
from sklearn.metrics import precision_score
precision=precision_score(y_test,y_pred)
print(precision)

0.8186465970284824


Probando el modelo conla data de Test


In [35]:
test = pd.read_csv('./hospitalizaciones_test.csv')

In [36]:
test

,Available Extra Rooms in Hospital,Department,Ward_Facility_Code,doctor_name,staff_available,patientid,Age,gender,Type of Admission,Severity of Illness,health_conditions,Visitors with Patient,Insurance,Admission_Deposit
0,3,gynecology,F,Dr Olivia,10,2858,21-30,Female,Trauma,Moderate,Other,2,No,4749.640225
1,2,TB & Chest disease,E,Dr John,4,65657,51-60,Female,Trauma,Moderate,Asthama,2,Yes,4122.261324
2,2,radiotherapy,E,Dr John,7,45598,51-60,Other,Trauma,Moderate,None,3,No,5459.067882
3,2,gynecology,F,Dr Olivia,1,31132,21-30,Female,Trauma,Moderate,None,2,Yes,3987.033318
4,2,anesthesia,A,Dr John,7,125021,51-60,Female,Trauma,Extreme,Diabetes,2,No,4770.789695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,4,gynecology,F,Dr Sarah,2,43001,11-20,Female,Trauma,Minor,High Blood Pressure,3,No,4105.795901
89996,13,gynecology,F,Dr Olivia,8,85601,31-40,Female,Emergency,Moderate,Other,2,No,4631.550257
89997,2,gynecology,B,Dr Sarah,3,22447,11-20,Female,Emergency,Moderate,High Blood Pressure,2,No,5456.930075
89998,2,radiotherapy,A,Dr John,1,29957,61-70,Female,Trauma,Extreme,Diabetes,2,No,4694.127772


In [37]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
test['Age_rango']= label_encoder.fit_transform(test['Age'])
test['health_conditions-valor']= label_encoder.fit_transform(test['health_conditions'])
test['Ward_Facility_Code-valor']= label_encoder.fit_transform(test['Ward_Facility_Code'])
test=pd.get_dummies(test)
test=test.drop(['patientid'], axis=1)
test.columns

Index(['Available Extra Rooms in Hospital', 'staff_available',
       'Visitors with Patient', 'Admission_Deposit', 'Age_rango',
       'health_conditions-valor', 'Ward_Facility_Code-valor',
       'Department_TB & Chest disease', 'Department_anesthesia',
       'Department_gynecology', 'Department_radiotherapy',
       'Department_surgery', 'Ward_Facility_Code_A', 'Ward_Facility_Code_B',
       'Ward_Facility_Code_C', 'Ward_Facility_Code_D', 'Ward_Facility_Code_E',
       'Ward_Facility_Code_F', 'doctor_name_Dr Isaac', 'doctor_name_Dr John',
       'doctor_name_Dr Mark', 'doctor_name_Dr Nathan', 'doctor_name_Dr Olivia',
       'doctor_name_Dr Sam', 'doctor_name_Dr Sarah', 'doctor_name_Dr Simon',
       'doctor_name_Dr Sophia', 'Age_0-10', 'Age_11-20', 'Age_21-30',
       'Age_31-40', 'Age_41-50', 'Age_51-60', 'Age_61-70', 'Age_71-80',
       'Age_81-90', 'Age_91-100', 'gender_Female', 'gender_Male',
       'gender_Other', 'Type of Admission_Emergency',
       'Type of Admission_Trauma

In [38]:
Prep = clf.predict(test)

In [40]:
Prep=pd.DataFrame(Prep)

In [42]:
Prep=Prep.rename(columns={0:'pred'})

In [44]:
Prep.to_csv('giselamdp.csv',index=False)

Se prepara el Pipeline


In [59]:
# Importamos las librerías necesarias

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import tree

In [60]:
# Creamos el primer pipeline

pipe_lr = Pipeline([('scl', StandardScaler()),
			('pca', PCA(n_components=2)),
			('clf', LogisticRegression(random_state=42))])

In [62]:
# Creamos el tercer pipeline

pipe_dt = Pipeline([('scl', StandardScaler()),
			('pca', PCA(n_components=2)),
			('clf', tree.DecisionTreeClassifier(random_state=42))])

In [63]:
# Los guardamos en una lista

pipelines = [pipe_lr, pipe_dt]

In [64]:
# Hacemos un diccionario para fines organizativos

pipe_dict = {0: 'Regresión Logística',  1: 'Árbol de decisión'}

In [65]:
# Entrenamos

for pipe in pipelines:
	pipe.fit(X_train, y_train)

In [66]:
# Evaluamos

for idx, val in enumerate(pipelines):
	print('%s pipeline accuracy en test: %.3f' % (pipe_dict[idx], val.score(X_test, y_test)))

Regresión Logística pipeline accuracy en test: 0.622
